# SQL Operation in Pandas

This blog is meant to provide examples to perform query using Pandas like what I usually do in SQL. I have used pandas for a while but I have not tried to dig deeper so I got stuck to some simple SQL-like query when I was using methods of dataframe. More specifically when I am using a groupby object. Therefore I decided to sit down and follow some tutorial and play with Pandas in order to be familiar with dataframe manipulations. In this post I am using MovieLens dataset [here](#). Following are some tutorial I went through

- [Intro to pandas data structure](http://www.gregreda.com/2013/10/26/intro-to-pandas-data-structures/)
- [Pandas cookbook](https://github.com/jvns/pandas-cookbook)
- [Pandas official doc](http://pandas.pydata.org/pandas-docs/version/0.18.0/comparison_with_sql.html)

## Read the data

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
dataset_path = "../data/ml-100k/"

u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv(dataset_path+'u.user', sep='|', names=u_cols)

r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv(dataset_path+'u.data', sep='\t', names=r_cols)

g_cols = ['genre', 'genre_id']
genres = pd.read_csv(dataset_path+'u.genre', sep='|', names=g_cols)

m_cols = ['movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url'] + genres.genre.values.tolist()
movies = pd.read_csv(dataset_path+'u.item', sep='|', names=m_cols)

## Joining the table

We need to join two tables together to perform queries with more information we desire. we can do that using dataframe's merge method. By default the merge function will perform a inner join, it is no surprise that more options we can perform like, left join, right join, outer join.  If we want to perform a left join to users and ratings table, we may use:

```
pd.merge(users, ratings, on='user_id', how='left')
```

As I have decided the coloum names myself so we don't need to specify the key to be joined. let say we want to join the users, rating, movies together.

In [3]:
full_records = ratings.merge(users).merge(movies)
full_records.head(3)

,user_id,movie_id,rating,unix_timestamp,age,sex,occupation,zip_code,title,release_date,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,196,242,3,881250949,49,M,writer,55105,Kolya (1996),24-Jan-1997,...,0,0,0,0,0,0,0,0,0,0
1,305,242,5,886307828,23,M,programmer,94086,Kolya (1996),24-Jan-1997,...,0,0,0,0,0,0,0,0,0,0
2,6,242,4,883268170,42,M,executive,98101,Kolya (1996),24-Jan-1997,...,0,0,0,0,0,0,0,0,0,0


As we are not going to use that much columns, we may constrcut a dataframe with less columns.

In [4]:
selected_cols = np.arange(12)
reduced_records = full_records[selected_cols]
reduced_records.head(3)

,user_id,movie_id,rating,unix_timestamp,age,sex,occupation,zip_code,title,release_date,video_release_date,imdb_url
0,196,242,3,881250949,49,M,writer,55105,Kolya (1996),24-Jan-1997,NaN,http://us.imdb.com/M/title-exact?Kolya%20(1996)
1,305,242,5,886307828,23,M,programmer,94086,Kolya (1996),24-Jan-1997,NaN,http://us.imdb.com/M/title-exact?Kolya%20(1996)
2,6,242,4,883268170,42,M,executive,98101,Kolya (1996),24-Jan-1997,NaN,http://us.imdb.com/M/title-exact?Kolya%20(1996)


## Groupby

### Which 10 movies got the most ratings?

In [5]:
by_title = reduced_records.groupby('title')
by_title.size().sort_values(ascending=False).head(10)

title
Star Wars (1977)                 583
Contact (1997)                   509
Fargo (1996)                     508
Return of the Jedi (1983)        507
Liar Liar (1997)                 485
English Patient, The (1996)      481
Scream (1996)                    478
Toy Story (1995)                 452
Air Force One (1997)             431
Independence Day (ID4) (1996)    429
dtype: int64

### Which is the top ten rated movies?

The query I used here comdine few concepts in order to get the result I want. Let's break it down.

I used **`by_title.agg`** to find the size and the mean of rating of each movies. **`np.size`** and **`np.mean`** are Numpy funtions that I want to pass and get the result from the rating column.

```
top_rated = by_title.agg({
        'rating': [np.size, np.mean]
    })
```

As there are many moives had only gotten few ratings so I would like to exclude movies with less than 50 ratings so that the sample size could be large enough. **`top_rated.rating['size'] > 50`** actually returns a new array of only True/False values and the consept here is to make a logical operation between **`top_rated`** and **`top_rated.rating['size'] > 50`**. The **`top_rated[top_rated.rating['size'] > 50]`** simply return a new dataframe object with size>50. And then we sort the values by the mean

In [6]:
top_rated = by_title.agg({
        'rating': [np.size, np.mean]
    })
top_rated[top_rated.rating['size'] > 50].sort_values(by=('rating', 'mean'), ascending=False).head(10)

rating          
                                                     size      mean
title                                                              
Close Shave, A (1995)                                 112  4.491071
Schindler's List (1993)                               298  4.466443
Wrong Trousers, The (1993)                            118  4.466102
Casablanca (1942)                                     243  4.456790
Wallace & Gromit: The Best of Aardman Animation...     67  4.447761
Shawshank Redemption, The (1994)                      283  4.445230
Rear Window (1954)                                    209  4.387560
Usual Suspects, The (1995)                            267  4.385768
Star Wars (1977)                                      583  4.358491
12 Angry Men (1957)                                   125  4.344000

### How Star War was rated by each age group?

In [14]:
age_group_labels = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79']
age_group = pd.cut(reduced_records['age'], bins=range(0, 81, 10), right=False, labels=age_group_labels)
reduced_records['age_group'] = age_group

In [16]:
star_war_by_age = reduced_records[reduced_records.title == 'Star Wars (1977)'].groupby('age_group')
star_war_by_age.agg({
        'rating': [np.size, np.mean]
    })

rating          
            size      mean
age_group                 
0-9            1  3.000000
10-19         46  4.630435
20-29        230  4.413043
30-39        157  4.305732
40-49         90  4.166667
50-59         49  4.469388
60-69          9  4.000000
70-79          1  4.000000

### Which 10 movies got the highest rating for age group 20-29?

In [23]:
reduced_records[reduced_records.age_group=='20-29'].groupby('title').agg({
        'rating': [np.size, np.mean]
    })[top_rated.rating['size'] > 50].sort_values(by=('rating', 'mean'), ascending=False).head(10)

rating          
                                                     size      mean
title                                                              
Shawshank Redemption, The (1994)                      123  4.585366
Wallace & Gromit: The Best of Aardman Animation...     34  4.558824
Schindler's List (1993)                               109  4.467890
Casablanca (1942)                                      76  4.447368
Silence of the Lambs, The (1991)                      152  4.434211
Usual Suspects, The (1995)                            120  4.416667
Star Wars (1977)                                      230  4.413043
Wrong Trousers, The (1993)                             56  4.410714
Close Shave, A (1995)                                  51  4.392157
Good Will Hunting (1997)                               63  4.365079

How many movies have you watched in this list??? For me? Star Wars (1977), Good Will Hunting (1997) haha

There is also another way to have the view of mean rating for each age group of movies.

In [56]:
new_view = reduced_records.groupby(['title','age_group']).rating.mean().unstack().fillna(0)
new_view.head(10)

age_group,0-9,10-19,20-29,30-39,40-49,50-59,60-69,70-79
title,,,,,,,,
'Til There Was You (1997),0,1.000000,2.285714,4.000000,0.000000,0.000000,0.000000,0
1-900 (1994),0,0.000000,1.000000,3.666667,0.000000,0.000000,0.000000,0
101 Dalmatians (1996),0,3.545455,2.648649,3.088235,2.950000,2.333333,2.000000,0
12 Angry Men (1957),0,4.500000,4.230769,4.382353,4.500000,4.235294,4.400000,0
187 (1997),0,3.333333,3.222222,2.200000,2.500000,3.333333,0.000000,0
2 Days in the Valley (1996),0,3.000000,3.276596,3.000000,3.538462,3.250000,3.000000,0
"20,000 Leagues Under the Sea (1954)",0,3.250000,3.217391,3.578947,3.615385,3.750000,4.000000,4
2001: A Space Odyssey (1968),5,4.100000,3.924731,3.887500,4.093023,4.000000,4.285714,0
3 Ninjas: High Noon At Mega Mountain (1998),0,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0


## Closing

This is a short practice and I hope I would be more confitable to use pandas next time. I followed the tutorials provided at the begining and please feel free to go through all those as they provide more details and usage in Pandas.